In [1]:
import os
import sys
import numpy as np
import random
import re
import pandas as pd
from typing import Dict, Optional
from subprocess import CalledProcessError, check_output
from pathlib import Path
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

In [2]:
# export
def _isASCII(mthd: str) -> bool:
    """
    Check if the given method contains only ASCII characters. From https://stackoverflow.com/a/27084708/5768407.

    :param mthd: the method to verify contains only ASCII characters
    :returns: returns a boolean representing whether or not the given method contains only ASCII characters
    """
    try:
        mthd.encode(encoding="utf-8").decode("ascii")
    except UnicodeDecodeError:
        return False
    else:
        return True


def remove_non_ascii(df: pd.DataFrame, n: Optional[int] = None) -> pd.DataFrame:
    """
    Remove all methods that contain non-ascii characters from a given pandas dataframe, not in-place.

    :param df: the pandas dataframe containing each method to be beautified
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a new dataframe without methods that contain non-ascii characters
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df = df[df.code.apply(_isASCII)]

    return df

In [3]:
# export
def _beautify(mthd: str) -> str:
    """
    Beautifies a given method using uncrustify with the sun.cfg style, i.e., Oracle's style.

    :param mthd: the method to beautify
    :returns: returns a beautified version of the given method
    """
    # get path of icodegen
    #icodegen_path = Path(icodegen.__path__[0])

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path("./tmp")
    tmp_path.mkdir(parents=True, exist_ok=True)
    with open(tmp_path / "tmp.java", "w") as f:
        f.write(mthd)

    try:
        beaut_mthd = check_output(
            [
                "./uncrustify",
                "-c",
                 "./sun.cfg",
                "-f",
                tmp_path / "tmp.java",
            ]
        ).decode("utf-8")
    except CalledProcessError as e:
        # Exception thrown when the method is malformed, i.e, it is missing a curly brace
        beaut_mthd = e.output.decode("utf-8")

    return beaut_mthd


def beautify_code(df: pd.DataFrame, n: Optional[int] = None) -> pd.DataFrame:
    """
    Beautify the methods in a pandas dataframe using uncrustify with the sun.cfg style, i.e., Oracle's style, not in-place.

    :param df: the pandas dataframe containing each method to be beautified
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the methods beautified
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(_beautify)

    return df

In [4]:
BEAUT_MTHD = pd.DataFrame(["""public void setPipelines(java.util.Collection<Pipeline> pipelines) {
    if (pipelines == null) {
	this.pipelines = null;
	return;
    }
    this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(
	pipelines);
}
"""], columns=["code"])

df_beaut = beautify_code(BEAUT_MTHD)

assert BEAUT_MTHD.code.values[0] == df_beaut.code.values[0]

In [4]:
# export
# dicts of special tokens we are adding to the tokenizers so they do not get split

extra_tokens = {"<n>": "\n"}

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/_keywords.html
java_reserved_tokens = {
    "<abstract>": "abstract",
    "<assert>": "assert",
    "<boolean>": "boolean",
    "<break>": "break",
    "<byte>": "byte",
    "<case>": "case",
    "<catch>": "catch",
    "<char>": "char",
    "<class>": "class",
    "<const>": "const",
    "<continue>": "continue",
    "<default>": "default",
    "<do>": "do",
    "<double>": "double",
    "<else>": "else",
    "<enum>": "enum",
    "<extends>": "extends",
    "<final>": "final",
    "<finally>": "finally",
    "<float>": "float",
    "<for>": "for",
    "<goto>": "goto",
    "<if>": "if",
    "<implements>": "implements",
    "<import>": "import",
    "<instanceof>": "instanceof",
    "<int>": "int",
    "<interface>": "interface",
    "<long>": "long",
    "<native>": "native",
    "<new>": "new",
    "<package>": "package",
    "<private>": "private",
    "<protected>": "protected",
    "<public>": "public",
    "<return>": "return",
    "<short>": "short",
    "<static>": "static",
    "<strictfp>": "strictfp",
    "<super>": "super",
    "<switch>": "switch",
    "<synchronized>": "synchronized",
    "<this>": "this",
    "<throw>": "throw",
    "<throws>": "throws",
    "<transient>": "transient",
    "<try>": "try",
    "<void>": "void",
    "<volatile>": "volatile",
    "<while>": "while",
}

# from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/opsummary.html
java_operator_tokens = {
    "<=>": "=",
    "<+>": "+",
    "<->": "-",
    "<*>": "*",
    "</>": "/",
    "<%>": "%",
    "<++>": "++",
    "<-->": "--",
    "<!>": "!",
    "<==>": "==",
    "<!=>": "!=",
    "<greater>": ">",
    "<greater_equal>": ">=",
    "<lesser>": "<",
    "<lesser_equal>": "<=",
    "<&&>": "&&",
    "<||>": "||",
    "<?>": "?",
    "<:>": ":",
    "<~>": "~",
    "<double_lesser>": "<<",
    "<double_greater>": ">>",
    "<triple_greater>": ">>>",
    "<&>": "&",
    "<^>": "^",
    "<|>": "|",
}

java_structural_tokens = {
    "<{>": "{",
    "<}>": "}",
    "<[>": "[",
    "<]>": "]",
    "<lesser>": "<",
    "<greater>": ">",
    "<(>": "(",
    "<)>": ")",
    "<;>": ";",
}

java_extra_tokens = {
    "<@>": "@",
    "<...>": "...",
    "<null>": "null",
    "<true>": "true",
    "<false>": "false",
}

# combination of all dictionaries
java_special_tokens = {
    **java_reserved_tokens,
    **java_operator_tokens,
    **java_structural_tokens,
    **java_extra_tokens,
    **extra_tokens,
}

In [31]:
# hide
df_fake = pd.DataFrame([">>> > + public ++ \n\n \t \t \t\t  "], columns=["code"])
df_fake

,code
0,>>> > + public ++ \n\n \t \t \t\t


In [5]:
# export
def _replace_toks(mthd: str, spec_toks: Dict[str, str]) -> str:
    """
    Helper function for replacing all special tokens in a given method. This will replace longer special tokens first in order to not mistakenly breakup a special token that is part of a longer sequence. Adapted from https://stackoverflow.com/a/6117124/5768407 and https://stackoverflow.com/a/11753945/5768407

    :param mthd: the method to have its special tokens replaced
    :param spec_toks: a dictionary containing the special tokens to replace and the new tokens to replace them with
    :returns: returns the method with its special tokens replaced
    """
    # construct escaped versions of keys for running through regex
    spec_toks = dict(
        (re.escape(v), k)
        for k, v in sorted(
            java_special_tokens.items(), key=lambda x: len(x[1]), reverse=True
        )
    )
    # construct regex pattern for finding all special tokens in a method
    pattern = re.compile("|".join(spec_toks.keys()))
    # replace all special tokens in a method
    mthd = pattern.sub(lambda m: spec_toks[re.escape(m.group(0))], mthd)

    return mthd


def replace_special_tokens(
    df: pd.DataFrame, spec_toks: Dict[str, str], n: Optional[int] = None
) -> pd.DataFrame:
    """
    Replace all the special tokens in a pandas dataframe.

    :param df: the pandas dataframe containing each method to replace special tokens in
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the special tokens replaced
    """
    if n is None:
        n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda mthd: _replace_toks(mthd, spec_toks))

    return df

In [6]:
REPLACED_MTHD = "<triple_greater> <greater> <+> <public> <++> <n><n> \t \t \t\t  "
df_replaced = replace_special_tokens(df_fake, java_special_tokens)

assert REPLACED_MTHD == df_replaced.code.values[0]

NameError: name 'df_fake' is not defined

In [35]:
# export

def __replace_tokenizer_toks(code_snippet: str) -> str:
    """
    Function to replace special tokens introduced by the tokenizer/model (bos, eos, pad)
    :param code_snippet: String representing the code snippet
    :return: String containing the clean string
    """
    pattern = re.compile("|".join(["<pad>", "<sos>", "<eos>"]))
    
    clean_snippet = pattern.sub("", code_snippet)
    return clean_snippet

def replace_tokenizer_toks(df: pd.DataFrame, n: Optional[int]=None) -> pd.DataFrame:
    """
    Function to replreace
    :param df: Pandas DataFrame containing the collection of code snippets
    :return: Clean DataFrame
    """
    
    if n is None:
        n = len(df)
    df = df.iloc[:n].copy()
    df.code = df.code.apply(lambda snippet: __replace_tokenizer_toks(snippet))
    return df

In [7]:
# export
def train_tokenizer(
    df: pd.DataFrame,
    spec_toks: Dict[str, str],
    max_length: int,
    n: Optional[int] = None,
    vocab_sz: Optional[int] = 10000,
    min_freq: Optional[int] = 2,
    output: Optional[Path] = None,
) -> Tokenizer:
    """
    Train a ByteLevel BPE tokenizer on a given pandas dataframe. Code adapted from https://github.com/huggingface/tokenizers/tree/master/bindings/python.

    :param df: the pandas dataframe containing each method to have the tokenizer train on
    :param spec_toks: dict of special tokens to add to the tokenizers so they do not get split
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :param vocab_sz: the maximum vocabulary size of the trained tokenizer. Defaulted was selected from: Big Code != Big Vocabulary: Open-Vocabulary Models for Source Code
    :param min_freq: the minimum frequency a token has to occur to be considered
    :returns: returns a trained ByteLevel BPE tokenizer
    """
    if n is None:
        n = len(df)

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path("./tmp")
    tmp_path.mkdir(parents=True, exist_ok=True)
    with open(tmp_path / "tmp_tokenize.txt", "w") as f:
        f.write("\n".join(df.code.values[:n]))

    # initialize a tokenizer
    tokenizer = Tokenizer(models.BPE())

    # customize pre-tokenization and decoding
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
    tokenizer.decoder = decoders.ByteLevel()
    tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

    # train tokenizer with data in tmp file
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_sz,
        min_frequency=min_freq,
        special_tokens=["<pad>", "<sos>", "<eos>"] + list(spec_toks.keys()),
    )
    tokenizer.train([str(tmp_path / "tmp_tokenize.txt")], trainer=trainer)
    tokenizer.enable_padding(max_length=max_length, pad_token="<pad>")
    tokenizer.enable_truncation(max_length)

    # save tokenizer if output path given
    if output is not None:
        tokenizer.save(output, pretty=True)

    return tokenizer

In [8]:
dataset_file = open('./dataset.txt', 'r')
length = []
tokenized_codes = []
non_ascii_files = []
filename_wrong = []
for line in dataset_file.readlines()[1:]:
    filename = line.split(',')[0]
    try:
        java_file = open(filename,'r').read()
    except:
        filename_wrong.append(filename)
    df = pd.DataFrame([java_file], columns=["code"])
    df = beautify_code(df)
    df = remove_non_ascii(df)
    df = replace_special_tokens(df, java_special_tokens)
    try:
        tokenized_codes.append([df.code.values[0]])
        length.append(len(df.code.values[0].split()))
    except:
        non_ascii_files.append(filename)
    
    
    

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

...

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: option 'sp_cpp_lambda_paren' is deprecated; use 'sp_cpp_lambda_square_paren' instead
./sun.cfg:658: option 'sp_word_brace' is deprecated; did you want to use 'sp_type_brace_init_lst' instead?
Parsing: tmp/tmp.java as language JAVA
./sun.cfg:303: o

In [9]:
df = pd.DataFrame(tokenized_codes,columns=["code"])

In [47]:
df.to_csv("pd_dataset.csv")

In [48]:
df.to_csv("pd_dataset_index_f.csv", index=False)

In [62]:
max(length)

8537

In [49]:
non_ascii_files

['../projects/modeshape-d43535a6c2d6e0be37f36a17303ec417979bde06/modeshape-common/src/test/java/org/modeshape/common/text/TokenStreamTest.java',
 '../projects/modeshape-d43535a6c2d6e0be37f36a17303ec417979bde06/modeshape-common/src/test/java/org/modeshape/common/text/UrlEncoderTest.java',
 '../projects/modeshape-d43535a6c2d6e0be37f36a17303ec417979bde06/modeshape-schematic/src/test/java/org/modeshape/schematic/internal/document/JsonWriterTest.java',
 '../projects/modeshape-d43535a6c2d6e0be37f36a17303ec417979bde06/modeshape-jcr/src/test/java/org/modeshape/jcr/JcrQueryManagerTest.java',
 '../projects/Mapper-b16688ed00609a4288004787f6f3f6476c349e62/core/src/test/java/tk/mybatis/mapper/annotation/VersionTest.java',
 '../projects/Mapper-b16688ed00609a4288004787f6f3f6476c349e62/base/src/test/java/tk/mybatis/mapper/cache/CacheTest.java',
 '../projects/Mapper-b16688ed00609a4288004787f6f3f6476c349e62/base/src/test/java/tk/mybatis/mapper/generatedvalue/GeneratedValueTest.java',
 '../projects/Mappe

In [10]:
tokenizer = train_tokenizer(df, java_special_tokens, max_length=2000)




enable_padding(max_length=X) is deprecated, use enable_padding(length=X) instead


In [92]:
tokenizer.save("tokenizer.json")

In [11]:
java_file

'package org.jmeterplugins.protocol.http.control;\n\nimport static org.junit.Assert.assertFalse;\nimport static org.junit.Assert.assertTrue;\n\nimport org.junit.After;\nimport org.junit.AfterClass;\nimport org.junit.Before;\nimport org.junit.BeforeClass;\nimport org.junit.Test;\n\npublic class HttpSimpleTableControlTest {\n\n    public HttpSimpleTableControlTest() {\n    }\n\n    @BeforeClass\n    public static void setUpClass() {\n    }\n\n    @AfterClass\n    public static void tearDownClass() {\n    }\n\n    @Before\n    public void setUp() {\n    }\n\n    @After\n    public void tearDown() {\n    }\n\n    @Test\n    public void testCanRemove() throws Exception {\n        System.out.println("canRemove");\n        HttpSimpleTableControl instance = new HttpSimpleTableControl();\n        assertTrue(instance.canRemove());\n\n        instance.startHttpSimpleTable();\n        assertFalse(instance.canRemove());\n\n        instance.stopHttpSimpleTable();\n        assertTrue(instance.canRemo

In [12]:
encoded = tokenizer.encode("<sos>"+java_file)

In [15]:
for id in encoded.ids[:500]:
    print(id,end=' ')

1 730 303 105 3250 8937 105 5661 105 682 105 377 1074 118 187 187 259 188 442 303 105 446 105 480 105 227 810 118 187 259 188 442 303 105 446 105 480 105 227 609 118 187 187 259 303 105 446 105 1560 118 187 259 303 105 446 105 3537 118 187 259 303 105 446 105 710 401 160 118 187 259 303 105 446 105 710 401 2836 118 187 259 303 105 446 105 257 118 187 187 272 1643 317 2529 1902 1318 2250 257 188 182 187 187 201 270 271 2529 1902 1318 2250 257 99 100 188 182 187 201 188 184 187 187 201 188 123 710 401 2836 187 201 270 271 188 442 188 299 5594 99 100 188 182 187 201 188 184 187 187 201 188 123 3537 187 201 270 271 188 442 188 299 5837 99 100 188 182 187 201 188 184 187 187 201 188 123 710 401 160 187 201 270 271 188 299 2053 99 100 188 182 187 201 188 184 187 187 201 188 123 1560 187 201 270 271 188 299 3156 99 100 188 182 187 201 188 184 187 187 201 188 123 257 187 201 270 271 188 299 3002 216 4197 99 100 285 4661 600 188 182 187 1595 2171 105 632 105 496 417 1337 99 93 2586 4197 93 100 

In [81]:
dataset_file = open('./correct_dataset.txt', 'r')
token_size = []
for i,line in enumerate(dataset_file.readlines()[1:5]):
    filename = line.split(',')[0]
    java_file = open(filename,'r').read()
    
    encoded = tokenizer.encode("<sos>"+java_file)
    print(encoded)

Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [94]:
tokenizer.encode(java_file).ids

[106,
 101,
 187,
 188,
 101,
 2832,
 188,
 99,
 682,
 117,
 106,
 106,
 1111,
 105,
 1120,
 105,
 861,
 100,
 187,
 188,
 101,
 187,
 188,
 101,
 1266,
 646,
 306,
 1142,
 457,
 103,
 1199,
 438,
 105,
 107,
 188,
 99,
 1110,
 219,
 1249,
 93,
 100,
 118,
 187,
 188,
 101,
 832,
 750,
 748,
 1095,
 285,
 222,
 795,
 1221,
 441,
 1257,
 688,
 306,
 457,
 105,
 187,
 188,
 101,
 1032,
 750,
 1379,
 421,
 957,
 468,
 306,
 457,
 933,
 187,
 188,
 101,
 187,
 188,
 101,
 975,
 1077,
 117,
 106,
 106,
 1111,
 105,
 799,
 105,
 861,
 106,
 1287,
 106,
 1400,
 104,
 109,
 105,
 107,
 187,
 188,
 101,
 187,
 188,
 101,
 1409,
 1368,
 1073,
 1401,
 1411,
 277,
 1412,
 494,
 441,
 1404,
 103,
 1336,
 187,
 188,
 101,
 828,
 646,
 306,
 457,
 341,
 828,
 751,
 657,
 219,
 584,
 1084,
 93,
 1265,
 103,
 187,
 188,
 101,
 1256,
 1163,
 877,
 1263,
 982,
 1054,
 1254,
 103,
 1241,
 1255,
 277,
 1264,
 105,
 187,
 188,
 101,
 1128,
 306,
 457,
 188,
 401,
 306,
 1147,
 334,
 211,
 1247,
 1267,
 1248

In [84]:
tokenizer.get_vocab_size()

10000

In [87]:
dataset_file = open('./dataset.txt', 'r')

out = open("correct_dataset.txt", 'w')
out.write("test_class, runtime\n")
for line in dataset_file.readlines()[1:]:
    filename = line.split(',')[0]
    if filename not in non_ascii_files and filename not in filename_wrong:
        out.write(line)
out.close()
    
    
    

In [90]:
tokenizer.encode_batch(java_file)

[Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=2000, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_to

In [91]:
dataset_file = open('./dataset.txt', 'r')
length = []
tokenized_codes = []
non_ascii_files = []
filename_wrong = []
for line in dataset_file.readlines()[1:]:
    filename = line.split(',')[0]
    try:
        java_file = open(filename,'r').read()
    except:
        filename_wrong.append(filename)
    df = pd.DataFrame([java_file], columns=["code"])
    df = beautify_code(df)
    df = remove_non_ascii(df)
    df = replace_special_tokens(df, java_special_tokens)
    try:
        tokenized_codes.append([df.code.values[0]])
        length.append(len(df.code.values[0].split()))
    except:
        non_ascii_files.append(filename)

ModuleNotFoundError: No module named 'transformers'